In [1]:
import pandas as pd
import seaborn as sns
import os
from os import path
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

In [2]:
#Feature Selection: Random Forest - creates a number of decision trees (default: 100)

#wherein a sample of the dataframe is trained and an output is generated. The outputs of 

#all decision trees are aggregated and then, using a majority vote, the final output is

#produced.

In [3]:
df = pd.read_csv("../data/cleaned/cleaned_indicators-01-04-2013-01-01-2017.csv")

In [4]:
df.shape

(1372, 783)

In [5]:
df.head()

,Date,bitcoin-marketcap_raw,bitcoin-price_raw,bitcoin-transactions_raw,bitcoin-sentinusd_raw,bitcoin-transactionvalue_raw,bitcoin-mediantransactionvalue_raw,bitcoin-transactionfees_raw,bitcoin-median_transaction_fee_raw,bitcoin-confirmationtime_raw,...,price-btc-rsi-3,price-btc-rsi-7,price-btc-rsi-14,price-btc-rsi-30,price-btc-rsi-90,price-btc-roc-3,price-btc-roc-7,price-btc-roc-14,price-btc-roc-30,price-btc-roc-90
0,2013/04/01,1039517192,94.715,52572,127826531,2592.0,64.987,0.0777,0.0474,7.273,...,99.456,96.992,95.075,93.774,91.450,9.383,36.980,104.747,185.993,606.456
1,2013/04/02,1183101461,107.749,63095,257306533,4400.0,94.955,0.1050,0.0539,7.956,...,99.811,98.201,96.416,94.987,92.645,23.206,50.270,109.704,231.964,716.218
2,2013/04/03,1318189886,120.003,63766,252194497,4478.0,102.533,0.1330,0.0600,8.229,...,99.902,98.752,97.190,95.785,93.508,34.759,51.448,107.789,259.795,808.284
3,2013/04/04,1357824968,123.562,66738,226470029,4249.0,123.314,0.1410,0.0618,9.351,...,99.919,98.869,97.368,95.977,93.724,30.456,44.093,93.098,237.355,834.762
4,2013/04/05,1414392012,128.664,61215,227376421,4348.0,125.183,0.1200,0.0643,9.114,...,99.941,99.022,97.602,96.233,94.013,19.411,48.589,95.718,229.992,876.762


In [6]:
df['bitcoin-price_raw_1d'] = df.iloc[:,2].shift(-1)
df['bitcoin-price_raw_7d'] = df.iloc[:,2].shift(-7)
df['bitcoin-price_raw_30d'] = df.iloc[:,2].shift(-30)
df['bitcoin-price_raw_90d'] = df.iloc[:,2].shift(-90)

In [7]:
df.head()

,Date,bitcoin-marketcap_raw,bitcoin-price_raw,bitcoin-transactions_raw,bitcoin-sentinusd_raw,bitcoin-transactionvalue_raw,bitcoin-mediantransactionvalue_raw,bitcoin-transactionfees_raw,bitcoin-median_transaction_fee_raw,bitcoin-confirmationtime_raw,...,price-btc-rsi-90,price-btc-roc-3,price-btc-roc-7,price-btc-roc-14,price-btc-roc-30,price-btc-roc-90,bitcoin-price_raw_1d,bitcoin-price_raw_7d,bitcoin-price_raw_30d,bitcoin-price_raw_90d
0,2013/04/01,1039517192,94.715,52572,127826531,2592.0,64.987,0.0777,0.0474,7.273,...,91.450,9.383,36.980,104.747,185.993,606.456,107.749,163.561,124.662,89.829
1,2013/04/02,1183101461,107.749,63095,257306533,4400.0,94.955,0.1050,0.0539,7.956,...,92.645,23.206,50.270,109.704,231.964,716.218,120.003,195.397,111.049,85.608
2,2013/04/03,1318189886,120.003,63766,252194497,4478.0,102.533,0.1330,0.0600,8.229,...,93.508,34.759,51.448,107.789,259.795,808.284,123.562,178.500,102.694,83.204
3,2013/04/04,1357824968,123.562,66738,226470029,4249.0,123.314,0.1410,0.0618,9.351,...,93.724,30.456,44.093,93.098,237.355,834.762,128.664,115.000,105.040,81.424
4,2013/04/05,1414392012,128.664,61215,227376421,4348.0,125.183,0.1200,0.0643,9.114,...,94.013,19.411,48.589,95.718,229.992,876.762,134.114,92.446,113.464,78.461


In [8]:
#1d price change:1=price increase or unchanged, 0=otherwise
df.loc[df['bitcoin-price_raw_1d'] >= df['bitcoin-price_raw'], '1d price higher or unchanged?'] = 1
df.loc[df['bitcoin-price_raw_1d'] < df['bitcoin-price_raw'], '1d price higher or unchanged?'] = 0

#7d price change:1=price increase or unchanged, 0=otherwise
df.loc[df['bitcoin-price_raw_7d'] >= df['bitcoin-price_raw'], '7d price higher or unchanged?'] = 1
df.loc[df['bitcoin-price_raw_7d'] < df['bitcoin-price_raw'], '7d price higher or unchanged?'] = 0

#30d price change:1=price increase or unchanged, 0=otherwise
df.loc[df['bitcoin-price_raw_30d'] >= df['bitcoin-price_raw'], '30d price higher or unchanged?'] = 1
df.loc[df['bitcoin-price_raw_30d'] < df['bitcoin-price_raw'], '30d price higher or unchanged?'] = 0

#90d price change:1=price increase or unchanged, 0=otherwise
df.loc[df['bitcoin-price_raw_90d'] >= df['bitcoin-price_raw'], '90d price higher or unchanged?'] = 1
df.loc[df['bitcoin-price_raw_90d'] < df['bitcoin-price_raw'], '90d price higher or unchanged?'] = 0

In [9]:
df.head()

,Date,bitcoin-marketcap_raw,bitcoin-price_raw,bitcoin-transactions_raw,bitcoin-sentinusd_raw,bitcoin-transactionvalue_raw,bitcoin-mediantransactionvalue_raw,bitcoin-transactionfees_raw,bitcoin-median_transaction_fee_raw,bitcoin-confirmationtime_raw,...,price-btc-roc-30,price-btc-roc-90,bitcoin-price_raw_1d,bitcoin-price_raw_7d,bitcoin-price_raw_30d,bitcoin-price_raw_90d,1d price higher or unchanged?,7d price higher or unchanged?,30d price higher or unchanged?,90d price higher or unchanged?
0,2013/04/01,1039517192,94.715,52572,127826531,2592.0,64.987,0.0777,0.0474,7.273,...,185.993,606.456,107.749,163.561,124.662,89.829,1.0,1.0,1.0,0.0
1,2013/04/02,1183101461,107.749,63095,257306533,4400.0,94.955,0.1050,0.0539,7.956,...,231.964,716.218,120.003,195.397,111.049,85.608,1.0,1.0,1.0,0.0
2,2013/04/03,1318189886,120.003,63766,252194497,4478.0,102.533,0.1330,0.0600,8.229,...,259.795,808.284,123.562,178.500,102.694,83.204,1.0,1.0,0.0,0.0
3,2013/04/04,1357824968,123.562,66738,226470029,4249.0,123.314,0.1410,0.0618,9.351,...,237.355,834.762,128.664,115.000,105.040,81.424,1.0,0.0,0.0,0.0
4,2013/04/05,1414392012,128.664,61215,227376421,4348.0,125.183,0.1200,0.0643,9.114,...,229.992,876.762,134.114,92.446,113.464,78.461,1.0,0.0,0.0,0.0


In [10]:
df = df.fillna(0).astype({"1d price higher or unchanged?": "int32", "7d price higher or unchanged?": "int32", "30d price higher or unchanged?": "int32", "90d price higher or unchanged?": "int32"}) 

In [11]:
df["90d price higher or unchanged?"]

0       0
1       0
2       0
3       0
4       0
       ..
1367    0
1368    0
1369    0
1370    0
1371    0
Name: 90d price higher or unchanged?, Length: 1372, dtype: int32

In [12]:
df

,Date,bitcoin-marketcap_raw,bitcoin-price_raw,bitcoin-transactions_raw,bitcoin-sentinusd_raw,bitcoin-transactionvalue_raw,bitcoin-mediantransactionvalue_raw,bitcoin-transactionfees_raw,bitcoin-median_transaction_fee_raw,bitcoin-confirmationtime_raw,...,price-btc-roc-30,price-btc-roc-90,bitcoin-price_raw_1d,bitcoin-price_raw_7d,bitcoin-price_raw_30d,bitcoin-price_raw_90d,1d price higher or unchanged?,7d price higher or unchanged?,30d price higher or unchanged?,90d price higher or unchanged?
0,2013/04/01,1039517192,94.715,52572,127826531,2592.0,64.987,0.0777,0.0474,7.273,...,185.993,606.456,107.749,163.561,124.662,89.829,1,1,1,0
1,2013/04/02,1183101461,107.749,63095,257306533,4400.0,94.955,0.1050,0.0539,7.956,...,231.964,716.218,120.003,195.397,111.049,85.608,1,1,1,0
2,2013/04/03,1318189886,120.003,63766,252194497,4478.0,102.533,0.1330,0.0600,8.229,...,259.795,808.284,123.562,178.500,102.694,83.204,1,1,0,0
3,2013/04/04,1357824968,123.562,66738,226470029,4249.0,123.314,0.1410,0.0618,9.351,...,237.355,834.762,128.664,115.000,105.040,81.424,1,0,0,0
4,2013/04/05,1414392012,128.664,61215,227376421,4348.0,125.183,0.1200,0.0643,9.114,...,229.992,876.762,134.114,92.446,113.464,78.461,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1367,2016/12/28,15256653605,950.293,305230,1278359909,6419.0,167.849,0.3250,0.1900,8.780,...,29.705,57.164,965.624,0.000,0.000,0.000,1,0,0,0
1368,2016/12/29,15504767388,965.624,304455,1289635710,5645.0,165.106,0.3400,0.1930,9.600,...,31.848,59.635,950.955,0.000,0.000,0.000,0,0,0,0
1369,2016/12/30,15271009245,950.955,286661,1490135455,7317.0,156.321,0.3330,0.1900,9.172,...,28.898,55.497,952.778,0.000,0.000,0.000,1,0,0,0
1370,2016/12/31,15302156774,952.778,250125,947360373,5247.0,143.503,0.3140,0.1740,9.730,...,27.447,55.999,970.988,0.000,0.000,0.000,1,0,0,0
